In [1]:
'''
1.每个样本提取arousal和valence两个维度的特征，每个维度提取787个特征，共787*2=1574，保存至文件1.csv
2.用tsfresh中的的impute函数，将1574个特征中空值用median替代，再进行MinMaxScaler缩放到0-10之间
3.用tsfresh中的select_features筛选一遍特征，剩余305维，保存至文件2.csv
4.用自定义的correlationAnalysis函数进行相关性的特征筛选，剩余69维
5.用RFE筛选30维的特征，保存至文件3.csv
6.划分训练集与测试集，在随机森林中观察分类准确率
7.用五折交叉验证，在随机森林中观察分类准确率
'''
import pandas as pd
import numpy as np
#读取arousal数据
arousal = pd.read_csv('arousal.csv',index_col=0)
data_arousal=[]
for i in range(0,744):
    X=np.array(arousal.iloc[i].dropna(axis=0,how='any'))
    data_arousal.append(X)

#读取valence数据
valence = pd.read_csv('valence.csv',index_col=0)
data_valence=[]
for i in range(0,744):
    X=np.array(valence.iloc[i].dropna(axis=0,how='any'))
    data_valence.append(X)
'''
Blues:0-99(100)   Classical:100-215(116)  Country:216-320(105)  Electronic:321-412(92)
Folk:413-483(71)  Jazz:484-588(105)  Pop:589-666(78)  Rock:667-743(77)
'''
blues_arousal,blues_valence=data_arousal[0:100],data_valence[0:100]
classical_arousal,classical_valence=data_arousal[100:216],data_valence[100:216]
country_arousal,country_valence=data_arousal[216:321],data_valence[216:321]
electronic_arousal,electronic_valence=data_arousal[321:413],data_valence[321:413]
folk_arousal,folk_valence=data_arousal[413:484],data_valence[413:484]
jazz_arousal,jazz_valence=data_arousal[484:589],data_valence[484:589]
pop_arousal,pop_valence=data_arousal[589:667],data_valence[589:667]
rock_arousal,rock_valence=data_arousal[667:744],data_valence[667:744]

In [2]:
#reshape为tsfresh的输入格式Flat DataFrame，每个维度拼接为一列
data_1 = np.array(data_arousal).reshape(-1,1)
data_2 = np.array(data_valence).reshape(-1,1)
np.shape(data_1)

(44640, 1)

In [3]:
#构造id和标签genre两列
#id为0-743（大小为744*60）,表示竖排的时间序列的id
#genre为0-7,表示每个时间序列的流派类别
names=['blues','classical','country','electronic','folk','jazz','pop','rock']
i = 0
genre = []
id = []
n = 0
for name in names:
    data = name+'_arousal'
    the_data = locals()[data]
    for i in range(len(the_data)):
        genre.append(n)
    n = n+1
genre = pd.Series(genre)
for j in range(0,744):
    id.append([j]*60)
id = np.array(id).reshape(-1,1)
np.shape(id)

(44640, 1)

In [4]:
#将arousal和valence数据拼接到一个DataFrame中
df = pd.DataFrame(data_1,columns=['arousal'])
df['valence'] = data_2
df['id'] = id
df

,arousal,valence,id
0,-0.109386,-0.073341,0
1,-0.114942,-0.074661,0
2,-0.116413,-0.074077,0
3,-0.118613,-0.078154,0
4,-0.126457,-0.081588,0
...,...,...,...
44635,0.494153,0.288580,743
44636,0.495761,0.289080,743
44637,0.496499,0.287102,743
44638,0.500411,0.285324,743


In [28]:
valence

array([[-0.0733407 , -0.0746607 , -0.0740767 , -0.0781537 ],
       [-0.0815877 , -0.0808727 , -0.0836107 , -0.0821447 ],
       [-0.0816317 , -0.0838737 , -0.0851047 , -0.0871877 ],
       ...,
       [ 0.2885927 ,  0.2879347 ,  0.2855917 ,  0.2797927 ],
       [ 0.2768157 ,  0.2877912 ,  0.2881632 ,  0.2885796 ],
       [ 0.28908019,  0.2871017 ,  0.2853237 ,  0.2859647 ]])

In [5]:
from tsfresh import extract_features
extracted_features = extract_features(df, column_id="id", column_sort=None)
extracted_features.to_csv('1.csv')

Feature Extraction: 100%|██████████| 100/100 [00:07<00:00, 13.56it/s]


In [6]:
extracted_features

,arousal__variance_larger_than_standard_deviation,arousal__has_duplicate_max,arousal__has_duplicate_min,arousal__has_duplicate,arousal__sum_values,arousal__abs_energy,arousal__mean_abs_change,arousal__mean_change,arousal__mean_second_derivative_central,arousal__median,...,valence__permutation_entropy__dimension_5__tau_1,valence__permutation_entropy__dimension_6__tau_1,valence__permutation_entropy__dimension_7__tau_1,valence__query_similarity_count__query_None__threshold_0.0,"valence__matrix_profile__feature_""min""__threshold_0.98","valence__matrix_profile__feature_""max""__threshold_0.98","valence__matrix_profile__feature_""mean""__threshold_0.98","valence__matrix_profile__feature_""median""__threshold_0.98","valence__matrix_profile__feature_""25""__threshold_0.98","valence__matrix_profile__feature_""75""__threshold_0.98"
0,0.0,0.0,0.0,0.0,-11.851045,2.553518,0.004796,-0.002673,3.711983e-05,-0.164383,...,1.933898,2.263613,2.499463,NaN,1.022371,3.754443,1.508820,1.279426,1.160179,1.427858
1,0.0,0.0,0.0,0.0,-11.591216,2.347900,0.004686,-0.002437,8.453966e-05,-0.173634,...,3.260731,3.600712,3.773917,NaN,0.840480,4.493477,1.893996,1.511375,1.169253,1.986586
2,0.0,0.0,0.0,0.0,14.584336,3.732612,0.006761,0.001403,3.779759e-04,0.261933,...,3.025082,3.304426,3.459891,NaN,0.807783,4.207042,2.401866,2.488631,1.121734,3.562673
3,0.0,0.0,0.0,1.0,-14.172403,3.352895,0.001777,0.000198,-1.548103e-04,-0.235395,...,2.754038,3.183644,3.345824,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,22.577521,8.668848,0.004556,-0.002071,3.883017e-05,0.344691,...,3.817966,4.007333,3.988984,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,0.0,0.0,0.0,0.0,-3.659443,0.233946,0.004241,0.000636,3.146552e-07,-0.061507,...,3.204487,3.548515,3.783607,NaN,0.726297,4.062532,1.694599,1.193594,0.970206,2.403541
740,0.0,0.0,0.0,0.0,21.766001,7.903712,0.002332,0.000500,1.008621e-06,0.366253,...,2.901676,3.463386,3.706591,NaN,0.650737,3.019540,1.428822,1.398836,0.914050,1.859547
741,0.0,0.0,0.0,0.0,18.029776,5.558762,0.010363,-0.000226,1.275690e-04,0.308491,...,2.593641,3.058058,3.327802,NaN,0.724569,3.686037,1.735403,1.578911,1.171402,2.107639
742,0.0,0.0,0.0,1.0,-9.621569,1.545550,0.002077,0.000434,5.753112e-05,-0.158470,...,1.222877,1.493041,1.765523,NaN,1.061847,2.010689,1.346520,1.120475,1.092754,1.668460


In [7]:
import warnings
warnings.filterwarnings("ignore")
#数据集预处理
from sklearn.preprocessing import MinMaxScaler
from tsfresh.utilities.dataframe_functions import impute
#-inf用最小值替代,inf用最大值替代，NaN用median替代
impute(extracted_features)
#标准化
scaler = MinMaxScaler(feature_range=(0,10))
extracted_features_scaled = scaler.fit_transform(extracted_features)
extracted_features_scaled = pd.DataFrame(extracted_features_scaled,columns=extracted_features.columns)

In [8]:
#用tsfresh中的select_features筛选一遍特征
import warnings
warnings.filterwarnings("ignore")
from tsfresh import select_features
features_filtered = select_features(extracted_features_scaled, genre)
features_filtered.to_csv('2.csv')
features_filtered

,"valence__fft_coefficient__attr_""imag""__coeff_23","valence__fft_coefficient__attr_""angle""__coeff_23","valence__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""max""","valence__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4","valence__fft_coefficient__attr_""imag""__coeff_1","valence__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""max""","valence__linear_trend__attr_""rvalue""","valence__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","valence__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""mean""","valence__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""max""",...,"arousal__linear_trend__attr_""slope""","arousal__fft_coefficient__attr_""real""__coeff_14","arousal__fft_coefficient__attr_""imag""__coeff_24","arousal__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""mean""","arousal__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""mean""","arousal__fft_coefficient__attr_""imag""__coeff_2",arousal__first_location_of_minimum,"arousal__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4",arousal__last_location_of_minimum,"arousal__fft_coefficient__attr_""angle""__coeff_30"
0,3.475490,4.521398,1.078319,1.677770,0.245887,0.162827,0.174977,0.862716,0.167310,0.186963,...,2.401636,7.341056,2.370520,2.399144,2.399121,4.040048,6.779661,2.077911,6.779661,0.0
1,3.474287,3.769819,3.303382,3.739467,2.612275,0.772671,0.752801,3.126595,0.717260,0.705178,...,2.965276,7.596792,2.972163,2.959845,2.951623,2.963289,8.644068,3.196239,8.644068,0.0
2,2.936001,1.157350,5.679170,5.159070,6.138182,8.466415,8.681436,5.709680,8.789842,8.879570,...,6.466105,3.578435,4.377341,6.462529,6.433542,5.360600,3.050847,5.014610,3.050847,10.0
3,3.730649,4.118351,4.002145,4.934766,3.799124,2.887212,3.088901,4.078696,2.919949,2.305040,...,4.784645,5.151719,3.301428,4.775455,4.748469,4.651841,0.000000,4.354853,0.000000,0.0
4,4.191639,4.631984,4.796552,4.523549,4.029939,3.350754,2.750547,4.440282,1.861980,5.807610,...,2.682876,6.954822,2.350076,2.675449,2.670401,2.720505,7.118644,2.351443,7.118644,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,4.328025,4.702805,4.035212,4.189722,3.628711,0.197617,0.326492,3.796562,0.316890,0.305654,...,4.922919,5.312592,4.079002,4.913178,4.889256,5.027838,2.372881,4.467788,2.372881,10.0
740,5.026860,8.973490,5.560988,5.755938,4.797924,9.395561,9.397209,5.236726,9.471920,9.514788,...,4.670555,5.174770,3.852686,4.659102,4.638356,4.841413,0.847458,4.356678,0.847458,10.0
741,6.729586,8.564811,7.125866,5.217932,6.400320,8.804033,8.725046,6.915078,8.839498,8.886177,...,4.582529,3.805262,4.430782,4.578394,4.555510,5.733291,2.203390,3.954520,2.203390,10.0
742,5.020878,9.410367,5.949890,5.564994,5.549455,9.907391,9.917825,5.838049,9.911319,9.884506,...,4.719047,5.418745,3.700229,4.710597,4.683085,4.446647,0.338983,4.429086,0.338983,10.0


In [9]:
def correlationAnalysis(train_data, fea_name, thr):
    #计算相关系数矩阵，初始化，设定强相关阈值thr
    corr = np.corrcoef(train_data, rowvar=False)
    sign = np.zeros(np.shape(corr))
    count = np.zeros(len(corr))
    abandon_list = np.ones(len(corr))
    thr = thr  
    
    #统计各特征名下的相关系数超过阈值的次数，并排序
    for i in range(len(corr)):
        for j in range(len(corr)):
            if corr[j,i] >= thr:
                sign[j,i] = 1
        count[i] = sum(sign[:,i])
    sortlist = np.argsort(count)
    
    #选取sortlist中次数最大的特征，把与之有强相关的特征列入清除列表
    #然后更新计数向量count，更新排序向量sortlist
    step = 0
    while np.max(count) >= 2:
        index = sortlist[-1]   # 取数组最后一个元素，也就是次数最大的特征索引
        step = step + 1
        n = 0
        if abandon_list[index] != 0:   #已经被清除的特征就不参与了
            for j in range(len(corr)):
                if sign[j,index] == 1 and j != index:
                    abandon_list[j] = 0
                    count[j] = -1
                    n = n + 1
        #print('第%d步：这一步强相关特征最多的是特征：%s，位于第%d列，剔除与它强相关的特征%d个' 
        #      %(step, fea_name[index], index+1, n))
        
        #重新统计剩余特征中的强相关次数高低，更新sortlist
        for i in range(len(corr)):
            if abandon_list[i] != 0:
                count[i] = 0
                for j in range(len(corr)):
                    if abandon_list[j] != 0:
                        count[i] = count[i] + sign[j,i]
        sortlist = np.argsort(count)    
        
    
    #统计剩余特征个数
    feaNum = sum(abandon_list)
    #print('筛选以后，剩余特征为%d个' %(feaNum))    
    
    #把剩余特征写到一个新的csv文件里
    fea_index = np.array(np.where(abandon_list == 1))
    remain_data = train_data[:,fea_index[0,:]]
    remain_column = fea_name[fea_index[0,:]]   
    return remain_data, remain_column 

In [10]:
# 然后，进行相关性的特征筛选
new_train_data, new_fname = correlationAnalysis(np.array(features_filtered), features_filtered.columns, 0.8)
np.shape(new_train_data)

(744, 69)

In [11]:
#用RFE再筛选出50个特征
from sklearn import svm
from sklearn.feature_selection import RFE
svc = svm.LinearSVC()
rfe = RFE(estimator=svc, n_features_to_select=20, step=1)
new_features = rfe.fit_transform(new_train_data,genre)
ranking = rfe.ranking_

In [12]:
#新特征对应的特征名
new_feature_columns = []
for i in range(len(ranking)):
    if ranking[i]==1:
        new_feature_columns.append(features_filtered.columns[i])
new_features = pd.DataFrame(new_features,columns=new_feature_columns)
new_features.to_csv('3.csv')
new_features.head()        

,"valence__fft_coefficient__attr_""imag""__coeff_23","valence__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4","valence__fft_coefficient__attr_""imag""__coeff_1","valence__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""mean""","valence__fft_coefficient__attr_""real""__coeff_22","valence__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""max""",valence__mean_change,"valence__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.2","valence__fft_coefficient__attr_""imag""__coeff_5","valence__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.2","valence__fft_coefficient__attr_""imag""__coeff_7","valence__fft_coefficient__attr_""angle""__coeff_7",valence__time_reversal_asymmetry_statistic__lag_2,valence__time_reversal_asymmetry_statistic__lag_3,valence__max_langevin_fixed_point__m_3__r_30,valence__quantile__q_0.4,valence__c3__lag_3,valence__mean,"valence__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""","valence__fft_coefficient__attr_""imag""__coeff_19"
0,3.475490,7.204317,1.638494,1.981076,4.866863,5.711442,2.383325,0.720450,0.170449,5.985970,3.646535,3.292410,3.520423,4.536757,5.418008,4.732592,2.169404,2.126306,4.929982,7.718993
1,3.474287,4.995979,3.706742,2.456389,5.393261,5.708191,4.800113,0.667483,0.172117,6.046409,4.205751,3.556752,3.466327,0.082606,5.473088,4.739947,1.829998,2.308151,4.929982,7.427307
2,2.936001,5.538361,4.918727,3.982480,8.290892,5.743405,5.424709,1.304456,0.174744,4.610862,5.870775,6.213751,3.825323,0.678416,6.183333,4.780722,4.031467,5.264996,4.929982,4.028807
3,3.730649,4.866800,4.452088,2.533990,3.128606,5.731002,4.262359,1.037766,0.173952,5.288862,4.744313,6.275062,4.082435,0.004355,5.833577,4.756296,2.576176,4.337458,3.721319,5.663068
4,4.191639,5.871396,4.388802,2.095489,5.903428,5.738945,4.507449,1.560368,0.173295,6.398168,3.177857,3.530429,5.213044,0.629728,4.556616,4.772819,1.387507,2.589579,4.929982,7.132831


In [13]:
#划分训练集和测试集
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(new_features,genre,test_size=0.2, stratify = genre,random_state=10)

In [17]:
#使用随机森林分类
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200, max_depth=6, max_features=6,random_state=2)
rf.fit(x_train,y_train)
print("The accuracy of the classifier on the train set is ", (rf.score(x_train, y_train)))
print("The accuracy of the classifier on the test set is ", (rf.score(x_test, y_test)))

The accuracy of the classifier on the train set is  0.8605042016806723
The accuracy of the classifier on the test set is  0.2348993288590604


In [18]:
#五折交叉验证
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold ,cross_val_score
strKFold = StratifiedKFold(n_splits=5,shuffle=False,random_state=None)
rf = RandomForestClassifier(n_estimators=200, max_depth=6, max_features=6,random_state=2)
scores_rf = cross_val_score(rf,new_features,genre,cv=strKFold)
print("straitified cross validation scores:{}".format(scores_rf))
print("Mean score of straitified cross validation:{:.2f}".format(scores_rf.mean()))

straitified cross validation scores:[0.24161074 0.24161074 0.24161074 0.26174497 0.20945946]
Mean score of straitified cross validation:0.24
